In [1]:
import pymongo
from pymongo import MongoClient
from pymongo.write_concern import WriteConcern

In [2]:
# Connessione al server MongoDB
client1 = MongoClient('mongodb://localhost:27117')
client2 = MongoClient('mongodb://localhost:27117')

# Imposta la write concern per entrambe le sessioni
read_concern = pymongo.read_concern.ReadConcern("local")
write_concern = WriteConcern(w=1)

trans = pymongo.client_session.TransactionOptions(read_concern = read_concern, write_concern = write_concern)

# Avvia due sessioni
session1 = client1.start_session(default_transaction_options=trans)
session2 = client2.start_session(default_transaction_options=trans)

In [4]:

try:
    # Avvia la transazione per la sessione 1
    with session2.start_transaction(read_concern=read_concern, write_concern=write_concern) as s2, session1.start_transaction(read_concern=read_concern, write_concern=write_concern) as s1:
        # Ottieni la collezione nella sessione 1
        
        collection1 = session1.client['db_bank']['bank']
        collection2 = session2.client['db_bank']['bank']
        
        result1 = collection1.find_one({ 'id': 299  })
        print("Lettura in T1: ", result1)

        # Esegui l'operazione di modifica nella sessione 1
        collection1.update_one(
            {'_id': result1['_id']},
            {'$inc': {'balance': 100}}, session = session1
        )
        
         # Esegui l'operazione di lettura nella sessione 2
        result2 = collection2.find_one({ 'id': 299  }, session = session2)
        print("Lettura in T2: ",result2)
        

        session1.commit_transaction()
        print("Transazione 1 committata")

        # Esegui l'operazione di modifica nella sessione 2
        collection2.update_one(
            {'_id': result2['_id']},
            {'$inc': {'balance': 100}}, session = session2
        )
        
        # Commit della transazione per la sessione 2
        session2.commit_transaction()
        print("Transazione 2 committata")
        
    result2 = collection2.find_one({ 'id': 299  }, session = session2)
    print(result2)

except Exception as e:
    # Esegui il rollback delle transazioni in caso di errore
   # session1.abort_transaction()
   # session2.abort_transaction()
    collection2.update_one(
            {'_id': result2['_id']},
            {'$inc': {'balance': 100}}, session = session2
        )
    print("Transazione 2 committata")
    result2 = collection2.find_one({ 'id': 299  }, session = session2)
    print("Lettura finale ", result2)

finally:
    # Chiudi le sessioni
    session1.end_session()
    session2.end_session()


Lettura in T1:  {'_id': ObjectId('64a958a37db95684b99cf37f'), 'id': 299, 'name': 'Gaia', 'surname': 'Pellegrini', 'date_of_birth': '1/11/1985', 'bank': 'UBI Banca', 'balance': 700}
Lettura in T2:  {'_id': ObjectId('64a958a37db95684b99cf37f'), 'id': 299, 'name': 'Gaia', 'surname': 'Pellegrini', 'date_of_birth': '1/11/1985', 'bank': 'UBI Banca', 'balance': 700}
Transazione 1 committata
Transazione 2 committata
Lettura finale  {'_id': ObjectId('64a958a37db95684b99cf37f'), 'id': 299, 'name': 'Gaia', 'surname': 'Pellegrini', 'date_of_birth': '1/11/1985', 'bank': 'UBI Banca', 'balance': 900}
